# Лабораторная работа: Мета-обучение
## Шаги 5-6: Анализ мета-набора и визуализация

**Цель:** Проанализировать корреляции мета-признаков и визуализировать мета-набор

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from src.visualization.plots import MetaVisualizer
from src.visualization.correlation import CorrelationPlotter
from src.visualization.dimensionality_reduction import DimReductionPlotter

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загружаем мета-набор
try:
    meta_dataset = pd.read_csv('../results/meta_dataset/meta_dataset_complete.csv')
    print(f"Загружен мета-набор: {meta_dataset.shape[0]} строк, {meta_dataset.shape[1]} колонок")
    print(f"Классы (лучшие алгоритмы): {meta_dataset['best_algorithm'].unique()}")
except:
    print("Мета-набор не найден. Сначала выполните предыдущие шаги.")
    meta_dataset = None

### 5.1 Анализ корреляции мета-признаков

In [ ]:
if meta_dataset is not None:
    # Выделяем числовые признаки
    feature_cols = [c for c in meta_dataset.columns 
                   if c not in ['dataset_id', 'dataset_name', 'best_algorithm']]
    
    meta_features = meta_dataset[feature_cols].select_dtypes(include=[np.number])
    print(f"Анализируем {meta_features.shape[1]} числовых мета-признаков")
    
    # Вычисляем корреляционную матрицу
    corr_matrix = meta_features.corr()
    
    # Визуализируем
    plt.figure(figsize=(16, 14))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, cmap='RdBu_r', center=0,
                annot=True, fmt='.2f', square=True, linewidths=0.5)
    plt.title('Корреляционная матрица мета-признаков', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Находим пары с высокой корреляцией
if meta_dataset is not None:
    threshold = 0.95
    high_corr = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                high_corr.append({
                    'feature1': corr_matrix.columns[i],
                    'feature2': corr_matrix.columns[j],
                    'correlation': corr_matrix.iloc[i, j]
                })
    
    if high_corr:
        high_corr_df = pd.DataFrame(high_corr)
        print(f"Найдено {len(high_corr)} пар с корреляцией > {threshold}:")
        print(high_corr_df)
        
        # Визуализируем
        plt.figure(figsize=(10, max(6, len(high_corr) * 0.3)))
        colors = ['red' if x > 0 else 'blue' for x in high_corr_df['correlation']]
        plt.barh(range(len(high_corr_df)), high_corr_df['correlation'], color=colors, alpha=0.7)
        plt.yticks(range(len(high_corr_df)), 
                  [f"{row['feature1']}\n{row['feature2']}" for _, row in high_corr_df.iterrows()])
        plt.xlabel('Корреляция')
        plt.title(f'Пары признаков с |корреляцией| > {threshold}')
        plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        plt.tight_layout()
        plt.show()
        
        print("\nОбъяснение высоких корреляций:")
        print("""
        Высокая корреляция возникает, когда признаки описывают одно и то же свойство:
        - Например, n_instances и log_instances
        - Различные меры разброса могут быть линейно связаны
        - В таких случаях можно удалить один из пары признаков
        """)
    else:
        print(f"Нет пар с корреляцией > {threshold}")

### 5.2 Удаление сильно коррелированных признаков

In [ ]:
if meta_dataset is not None:
    # Выбираем признаки для удаления
    features_to_drop = set()
    
    for item in high_corr:
        # Удаляем второй признак из пары
        features_to_drop.add(item['feature2'])
    
    print(f"Признаки для удаления: {features_to_drop}")
    
    # Создаем отфильтрованный набор
    filtered_features = meta_features.drop(columns=list(features_to_drop))
    print(f"После фильтрации осталось {filtered_features.shape[1]} признаков")

### 6.1 PCA визуализация

In [ ]:
if meta_dataset is not None:
    # Подготовка данных
    X = filtered_features.fillna(filtered_features.mean()).values
    y = meta_dataset['best_algorithm'].values
    
    # Масштабирование
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    # Визуализация
    plt.figure(figsize=(12, 8))
    
    # Уникальные алгоритмы
    unique_algos = np.unique(y)
    colors = plt.cm.Set1(np.linspace(0, 1, len(unique_algos)))
    
    for i, algo in enumerate(unique_algos):
        mask = y == algo
        plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                   label=algo, color=colors[i], alpha=0.7, s=50)
    
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    plt.title('PCA проекция мета-набора')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Объясненная дисперсия: PC1={pca.explained_variance_ratio_[0]:.2%}, PC2={pca.explained_variance_ratio_[1]:.2%}")

### 6.2 t-SNE визуализация

In [ ]:
if meta_dataset is not None:
    # t-SNE с разными perplexity
    perplexities = [30, 50]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    for idx, perp in enumerate(perplexities):
        tsne = TSNE(n_components=2, perplexity=perp, random_state=42)
        X_tsne = tsne.fit_transform(X_scaled)
        
        for i, algo in enumerate(unique_algos):
            mask = y == algo
            axes[idx].scatter(X_tsne[mask, 0], X_tsne[mask, 1],
                            label=algo, color=colors[i], alpha=0.7, s=50)
        
        axes[idx].set_xlabel('t-SNE1')
        axes[idx].set_ylabel('t-SNE2')
        axes[idx].set_title(f't-SNE (perplexity={perp})')
        axes[idx].grid(True, alpha=0.3)
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Анализ визуализации
print("\nАнализ визуализации:")
print("""
1. PCA проекция:
   - Позволяет увидеть общую структуру данных
   - Сохраняет глобальные расстояния
   - Хорошо для понимания дисперсии

2. t-SNE проекция:
   - Лучше показывает локальные кластеры
   - Чувствителен к параметру perplexity
   - Помогает увидеть разделение классов

3. Интерпретация:
   - Если точки одного цвета группируются вместе - мета-признаки хорошо разделяют задачи
   - Перекрытие кластеров указывает на сложность предсказания
""")

### Выводы по шагам 5-6

In [ ]:
print("="*60)
print("ВЫВОДЫ ПО ШАГАМ 5-6: АНАЛИЗ МЕТА-НАБОРА")
print("="*60)
print(f"""
1. КОРРЕЛЯЦИОННЫЙ АНАЛИЗ:
   - Исходное количество признаков: {meta_features.shape[1]}
   - Найдено {len(high_corr)} пар с высокой корреляцией
   - После фильтрации осталось: {filtered_features.shape[1]} признаков

2. ВИЗУАЛИЗАЦИЯ:
   - PCA объясняет {pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1]:.2%} дисперсии
   - t-SNE показывает {'хорошее' if len(unique_algos) <= 3 else 'умеренное'} разделение классов

3. ГОТОВНОСТЬ К МЕТА-ОБУЧЕНИЮ:
   - Мета-набор очищен от избыточных признаков
   - Визуализация подтверждает информативность признаков
   - Можно переходить к обучению мета-алгоритмов
""")